In [5]:
import pandas as pd
import Revisions
from time import time
import numpy as np
import re
from datetime import date, timedelta
from os.path import isfile
import pymysql
import matplotlib.pyplot as plt
import Revisions
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_ep = pd.read_csv('070916_edit_protected_articles.csv')
df_nep = pd.read_csv('../Sampling/071816_batch1_sampled_ids.csv')

In [3]:
start = 0
end = len(df_ep['page_id'].values)
#revs = [Revisions.GetRevisions(df_ep['page_id'].values[i]) for i in range(start, end)]

In [22]:
t0 = time()
dfs = [Revisions.RevisionToWeeklyAgg(revs[i]) if revs[i] is not None else None for i in range(start, end)]
print(time() - t0)

467.97453117370605


In [23]:
path1 = 'RevisionsOverTime.csv'
path2 = 'Proections.csv'
for item in dfs:
    if item is not None:
        header = not isfile(path1)
        item[0].to_csv(path1, mode='a', header=header, index=False)
        item[1].to_csv(path2, mode='a', header=header, index=False)

In [4]:
path1 = 'RevisionsOverTime_ep_V6.csv'
path2 = 'Proections_ep_V6.csv'
i = 0
while True:
    t0 = time()
    if i + 1000 < len(df_ep['page_id'].values):
        Revisions.WriteRevsToFile(df_ep['page_id'].values, i, i + 1000, path1, path2)
        i = i + 1000
    else:
        Revisions.WriteRevsToFile(df_ep['page_id'].values, i, len(df_ep['page_id'].values), path1, path2)
        break
    print(time() - t0)

914.6709489822388
743.4928541183472
673.226371049881
617.0911591053009
655.1088061332703
667.0097079277039
614.1662881374359


In [8]:
path1 = 'RevisionsOverTime_nep_Sample1.csv'
path2 = 'Proections_nep_Sample1.csv'
i = 0
while True:
    t0 = time()
    if i + 1000 < len(df_nep['page_id'].values):
        Revisions.WriteRevsToFile(df_nep, i, i + 1000, path1, path2)
        i = i + 1000
    else:
        Revisions.WriteRevsToFile(df_nep, i, len(df_nep['page_id'].values), path1, path2)
        break
    print(time() - t0)

619.9270131587982
600.0186240673065
696.2238388061523
706.3109328746796
638.3793778419495
637.9818201065063
631.2505769729614
537.2403650283813
564.6604261398315
641.1110799312592


In [6]:
creds = {'user': 'kevin', 
         'pswd': 'osullivan'} 


atriskwikiconn = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

In [7]:
cur = atriskwikiconn.cursor()

In [8]:
for batch in range(1, 9):
    print(batch)
    cur.execute("SELECT page_id FROM core WHERE page_batch = %d;" % batch)
    data = cur.fetchall()
    data = [item[0] for item in data]
    path1 = 'RevisionsOverTime_nep_Sample%d_V6.csv' % batch
    path2 = 'Proections_nep_Sample%d_V6.csv' % batch
    i = 0
    while True:
        t0 = time()
        if i + 1000 < len(data):
            Revisions.WriteRevsToFile(data, i, i + 1000, path1, path2)
            i = i + 1000
        else:
            Revisions.WriteRevsToFile(data, i, len(data), path1, path2)
            break
        print(time() - t0)

1
530.7052299976349
481.672837972641
469.97899198532104
515.2236320972443
481.5783450603485
470.2797360420227
466.54389119148254
463.6826181411743
444.4895329475403
2
499.017294883728
472.7494750022888
462.3438138961792
460.7908630371094
454.74826407432556
452.3596329689026
447.561527967453
443.17200994491577
3
502.5242929458618
479.757443189621
474.54514384269714
482.8369131088257
452.5741250514984
450.12632393836975
449.0543761253357
447.0975329875946
446.3962140083313
4
556.7018671035767
519.4286870956421
526.8034279346466
569.348659992218
546.9274079799652
554.7628099918365
573.8924160003662
558.5083048343658
548.8497231006622
5
577.7293810844421
542.033252954483
551.488734960556
561.9272110462189
508.2713589668274
503.3398640155792
471.04532289505005
448.6639709472656
447.4896328449249
6
506.63947200775146
524.0363531112671
472.84568905830383
454.4540250301361
452.1773691177368
448.70884108543396
444.2672870159149
469.4681041240692
455.3660321235657
7
575.9878959655762
527.5757069

In [8]:
print(i)

8000


In [19]:
cur.execute("SELECT page_title, page_id FROM core WHERE page_batch = %d;" % 2)
data = cur.fetchall()

In [21]:
page_titles = [item[0] for item in data]
page_ids = [item[1] for item in data]